In [ ]:
#set this to get small dataset
#directory : everything goes to this directory: storage/dataset
#manually rename the folder for large and dataset_small and dataset_large
small=True

In [ ]:
import sys
sys.path.append("~/PycharmProjects/code-trans-aug/")
dr = '~/PycharmProjects/code-trans-aug/storage/'
print('ANTLR features required for this part\nsetting problem class in antlr features')
import pandas as pd
column_names = ['f_loc', 'variable', 'arguments', 'operators', 'expressions', 'loops','operands', 'exceptions', 'exception_clause', 'mccable_complex' ]
or_java_feat = pd.read_csv(dr+'feat_original/java_features.csv', names=column_names)
or_py_feat = pd.read_csv(dr+'feat_original/python_features.csv', names=column_names)
tr_java_feat = pd.read_csv(dr+'feat_trans/java_features.csv', names=column_names)
tr_py_feat = pd.read_csv(dr+'feat_trans/python_features.csv', names=column_names)


In [ ]:
def filter_code(df):
    cp = 1
    cj = 1
    refined = []
    for index, row in df.iterrows():
        if row.language_extension =='py':
            if(len(row.code.split('\n'))>6):
                cp+=1
                refined.append(list(row))
        if row.language_extension =='java':
            if(len(row.code.split(';'))>6):
                cj+=1
                refined.append(list(row))
    print(df[df.language_extension =='py'].shape[0]+ df[df.language_extension =='java'].shape[0], pd.DataFrame(refined).shape, (cp+cj)/df.shape[0])
    return pd.DataFrame(refined, columns=list(df.columns))


In [ ]:
def set_class_label(df, id_):
    pc = []
    fp = []
    #uid =[]
    for i in range(len(df)):
        p = df.iloc[i]['f_loc'].split('/')[7:-1]
        s = "/".join(p)
        q = df.iloc[i]['f_loc'].split('/')[7:]
        t = "/".join(q)
        #uid.append(id_)
        #id_= id_ + 1
        fp.append(t)
        pc.append(s)
        
        
    df['file_path'] = fp
    df['problem_class'] = pc
    #df['id'] = uid
    
    return id_
    
id_ = set_class_label(or_java_feat, 1) #send each feature csv as argument
id_ = set_class_label(or_py_feat, id_)
id_ = set_class_label(tr_java_feat, id_)
id_ = set_class_label(tr_py_feat, id_)  

or_java_feat_pc = or_java_feat.iloc[: , 1:]
or_py_feat_pc = or_py_feat.iloc[: , 1:]

tr_java_feat_pc = tr_java_feat.iloc[: , 1:]
tr_py_feat_pc =tr_py_feat.iloc[: , 1:]



In [ ]:
print(or_java_feat_pc.shape,or_py_feat_pc.shape, tr_java_feat.shape, tr_py_feat.shape)

In [ ]:
ofp = pd.read_pickle(dr+'data_original.pkl').sample(frac=.7) #all fragments original dataset
ojava = ofp[ofp['language_extension']=='java']
opy = ofp[ofp['language_extension']=='py']

tfp = pd.read_pickle(dr+'data_transpiled.pkl')#all transpiled 
tjava = tfp[tfp['language_extension']=='java']
tpy = tfp[tfp['language_extension']=='py']

print(ofp.shape, tfp.shape, ofp.shape[0]-tfp.shape[0])

In [ ]:
ofp

In [ ]:
def set_id(temp, from_df):
    uid = []
    for i in range(len(temp)):
        f = temp.iloc[i]['file_path']
        k = from_df[from_df['file_path']==f].iloc[0]['id']
        uid.append(k)
    temp['id'] = uid
    return temp

In [ ]:
temp = or_java_feat_pc[or_java_feat_pc['file_path'].isin(ojava['file_path'])]
temp = set_id(temp, ojava)
temp.to_csv(dr+'feat_original/java_features_pc_id.csv', index=False, header=None)
java_features_pc_id = temp
temp.shape

In [ ]:
temp = or_py_feat_pc[or_py_feat_pc['file_path'].isin(opy['file_path'])]
temp = set_id(temp, opy)
temp.to_csv(dr+'feat_original/python_features_pc_id.csv', index=False, header=None)
python_features_pc_id = temp
temp.shape

In [ ]:
temp = tr_java_feat_pc[tr_java_feat_pc['file_path'].isin(tjava['file_path'])]
temp = set_id(temp, tjava)
temp.to_csv(dr+'feat_trans/java_features_pc_id.csv', index=False, header=None)
trans_java_features_pc_id = temp
temp.shape

In [ ]:
temp = tr_py_feat_pc[tr_py_feat_pc['file_path'].isin(tpy['file_path'])]
temp = set_id(temp, tpy)
temp.to_csv(dr+'feat_trans/python_features_pc_id.csv', index=False, header=None)
trans_python_features_pc_id = temp
temp.shape

In [ ]:

#augmented: merged dataset(or+tr)

ojava = pd.read_csv(dr+'feat_original/java_features_pc_id.csv', header=None)
opy = pd.read_csv(dr+'feat_original/python_features_pc_id.csv', header=None)

tjava = pd.read_csv(dr+'feat_trans/java_features_pc_id.csv', header=None)
tpy = pd.read_csv(dr+'feat_trans/python_features_pc_id.csv', header=None)

ojava.append(tjava).to_csv(dr+'feat_aug/pair_creation_augmented_java.csv', header=None, index=False)
opy.append(tpy).to_csv(dr+'feat_aug/pair_creation_augmented_py.csv', header=None, index=False)

In [ ]:
originals_ids_clc = list(python_features_pc_id.id) + list(java_features_pc_id.id)
trans_ids_clc = list(trans_python_features_pc_id.id) + list(trans_java_features_pc_id.id)
len(originals_ids_clc), len(trans_ids_clc)

In [ ]:
12 in originals_ids_clc

In [ ]:
import pandas as pd
dr = '~/PycharmProjects/code-trans-aug/storage/'

original = pd.read_pickle(dr+'data_original.pkl')
print(original.shape)
trans = pd.read_pickle(dr+'data_transpiled.pkl')
print(trans.shape)

# filter to get small dataset

In [ ]:
if small==True:
    original = filter_code(original)
    trans = filter_code(trans)

In [ ]:
original = original[original.id.isin(originals_ids_clc)]
trans = trans[trans.id.isin(trans_ids_clc)]

print(original.shape)
print(trans.shape)
print(original.is_translated.unique(), trans.is_translated.unique())

In [ ]:
#original GMN pairs : java and python
# input: java and java prime [original python to java prime] [parsable and all]

#augmented GMN pairs: java+python_to_java_prime and python+java_to_python_prime
#input : java+java' and convert to java(python and pythone prime both (this case python is double converted))
# merged dataset(or+tr)

#python : python and python prime[original java to python prime]
#AUG : original java ; original python cnv java(same as java prime) + python prime cnv java, java prime

In [ ]:
java_original = original[original.language_extension=='java']
print(java_original.shape)
trans = trans.sort_values(['id'])
#python original converted for gmn input
python_original_cnv_jv = trans[trans.language_extension=='java']
print(python_original_cnv_jv.shape)
python_original_cnv_jv['id'] = [k-38110 for k in list(python_original_cnv_jv.id)]
python_original_cnv_jv.language_extension = 'py'
print(python_original_cnv_jv.shape)

gmn_original = pd.concat([java_original, python_original_cnv_jv])

In [ ]:
gmn_original.shape, original.shape

In [ ]:
gmn_original = gmn_original[gmn_original.id.isin(original.id)]
print(gmn_original.shape)
original = original[original.id.isin(gmn_original.id)]
print(original.shape)

In [ ]:
12 in original.id

In [ ]:
python_to_java_prime = trans[trans.language_extension=='java']
print(python_to_java_prime.shape)
#trans[trans.language_extension=='py'].sort_values(['file_path'])
#java_to_python_prime is converted as following
python_prime_to_java = pd.read_pickle('~/PycharmProjects/code-trans-aug/storage/python_prime_to_java.pkl')
python_prime_to_java.sort_values(['file_path'])
python_prime_to_java = python_prime_to_java[python_prime_to_java.id.isin(trans.id)]
print(
    list(python_prime_to_java.sort_values(['file_path']).file_path) == 
    list(trans[trans.language_extension=='py'].sort_values(['file_path']).file_path)
)
print(python_prime_to_java.shape)
python_prime_to_java = python_prime_to_java[python_prime_to_java.language_extension!='error']
print(python_prime_to_java.shape)

In [ ]:
gmn_transpiled = pd.concat([python_prime_to_java, python_to_java_prime])

In [ ]:
gmn_transpiled.shape, trans.shape

In [ ]:
trans = trans[trans.id.isin(gmn_transpiled.id)]

In [ ]:
gmn_transpiled.shape, trans.shape

In [ ]:
gmn_transpiled = gmn_transpiled[gmn_transpiled.id.isin(trans.id)]

In [ ]:
java_original.shape, python_original_cnv_jv.shape

In [ ]:
python_prime_to_java.shape, python_to_java_prime.shape

In [ ]:
gmn_original.shape, original.shape, gmn_transpiled.shape, trans.shape

In [ ]:
original.sort_values(['id'], inplace=True)
gmn_original.sort_values(['id'], inplace=True)

trans.sort_values(['id'], inplace=True)
gmn_transpiled.sort_values(['id'], inplace=True)


In [ ]:
list(gmn_original.id) == list(original.id),list(gmn_original.problem_class) == list(original.problem_class)


In [ ]:
list(gmn_transpiled.id) == list(trans.id), list(gmn_transpiled.problem_class) == list(trans.problem_class)

In [ ]:
#original.to_pickle('original_data_final.pkl')
codes = pd.concat([original, trans])
gmn_codes = pd.concat([gmn_original, gmn_transpiled])
mapping = {item:i+1 for i, item in enumerate(codes["problem_class"].unique())}
codes["task"] = codes['problem_class'].apply(lambda x: mapping[x])
gmn_codes["task"] = gmn_codes['problem_class'].apply(lambda x: mapping[x])


In [ ]:
list(gmn_codes.task) == list(codes.task)

In [ ]:
codes.to_pickle(dr+'dataset/codes.pkl')
codes.shape

In [ ]:
print(len(set(gmn_codes.id)) == gmn_codes.shape[0])
gmn_codes_dict = dict(gmn_codes[['id', 'code']].values)
df = pd.DataFrame(gmn_codes_dict.items())
df.to_pickle(dr+'dataset/gmn_codes_dict.pkl')

In [ ]:
def get_map(df):
    df["task"] = df['problem_class'].apply(lambda x: mapping[x])
    return df

original = get_map(original)
original.to_pickle(dr+'dataset/original_final.pkl')
trans = get_map(trans)
trans.to_pickle('dataset/transpiled_final.pkl')

gmn_original = get_map(gmn_original)
gmn_original.to_pickle(dr+'dataset/gmn_original_final.pkl')
gmn_transpiled = get_map(gmn_transpiled)
gmn_transpiled.to_pickle('dataset/gmn_transpiled_final.pkl')

In [ ]:
list(gmn_original.language_extension) == list(original.language_extension)

In [ ]:
list(trans.language_extension) == list(gmn_transpiled.language_extension)

In [ ]:
def  filter_for_pair (df):
    df = df[['id', 'task']]
    return df

In [ ]:
original_java = original[original.language_extension=='java']
original_java = filter_for_pair(original_java)
original_java.to_csv('dataset/java_original.csv', header=None, index=False)

original_python = original[original.language_extension=='py']
original_python = filter_for_pair(original_python)
original_python.to_csv('dataset/python_original.csv', header=None, index=False)

In [ ]:
trans_java = trans[trans.language_extension=='java']
pd.concat([filter_for_pair(trans_java), original_java]).to_csv('dataset/java_aug.csv', header=None, index=False)

trans_python = trans[trans.language_extension=='py']
pd.concat([filter_for_pair(trans_python), original_python]).to_csv('dataset/python_aug.csv', header=None, index=False)

### Run Code inside Java Runner to get the pairs first.

- now read clones and non clones
- merge pairs
- sample

In [ ]:
import pandas as pd
dr = '~/PycharmProjects/code-trans-aug/storage/'
clones =pd.read_csv(dr+'dataset/clones.csv', header=None)
print(clones.shape)

non_clones =pd.read_csv(dr+'dataset/non_clones.csv', header=None).sample(clones.shape[0])
print(non_clones.shape)

In [ ]:
ag_clones =pd.read_csv(dr+'dataset/clones_augmented.csv', header=None)
print(ag_clones.shape)


In [ ]:
ag_non_clones =pd.read_csv(dr+'dataset/non_clones_augmented.csv', header=None).sample(ag_clones.shape[0])
print(ag_non_clones.shape)

In [ ]:
if small==True:
    do = pd.concat([clones, non_clones]).sample(frac=.6)
    do.to_csv(dr+'dataset/dataset_org.csv', header=None, index=False)

    da = pd.concat([ag_clones, ag_non_clones])
    da.to_csv(dr+'dataset/dataset_aug.csv', index=False)

    dataset_or_80_20_aug = pd.concat([do, da.sample(int(len(do)*0.25))])
    dataset_or_80_20_aug = dataset_or_80_20_aug.to_csv(dr+'dataset/dataset_or_80_20_aug.csv', header=None, index=False)
else:
    do = pd.concat([clones, non_clones])
    do.to_csv(dr+'dataset/dataset_org.csv', header=None, index=False)

    da = pd.concat([ag_clones, ag_non_clones])
    da.to_csv(dr+'dataset/dataset_aug.csv', index=False)

    dataset_or_80_20_aug = pd.concat([do, da.sample(int(len(do)*0.25))])
    dataset_or_80_20_aug = dataset_or_80_20_aug.to_csv(dr+'dataset/dataset_or_80_20_aug.csv', header=None, index=False)